In [2]:
from data.helper import *

G:\AppData\Anaconda\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
import torch, random
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl

In [4]:
users = load_users()

In [5]:
users.head()

,UserID,Gender,Age,Occupation,Zipcode
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [ ]:
df['Zipcode'] = df['zip_code'].astype(str)

# Extract first two digits and find number of unique ones
df['zip_prefix'] = df['zip_code'].str[:2]
num_unique_prefixes = df['zip_prefix'].nunique()